In [2]:
import os
import io
import numpy
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

def readFiles(path):
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            path = os.path.join(root, filename)

            inBody = False
            lines = []
            f = io.open(path, 'r', encoding='latin1')
            for line in f:
                if inBody:
                    lines.append(line)
                elif line == '\n':
                    inBody = True
            f.close()
            message = '\n'.join(lines)
            yield path, message


def dataFrameFromDirectory(path, classification):
    rows = []
    index = []
    for filename, message in readFiles(path):
        rows.append({'message': message, 'class': classification})
        index.append(filename)

    return DataFrame(rows, index=index)


In [3]:

data = DataFrame({'message': [], 'class': []})



In [4]:
data = data.append(dataFrameFromDirectory('emails/spam', 'spam'))
data = data.append(dataFrameFromDirectory('emails/ham', 'ham'))


In [5]:
data

,message,class
emails/spam\00001.7848dde101aa985090474a91ec93fcf0,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 Tr...",spam
emails/spam\00002.d94f1b97e48ed3b553b3508d116e6a09,1) Fight The Risk of Cancer!\n\nhttp://www.adc...,spam
emails/spam\00003.2ee33bc6eacdb11f38d052c44819ba6c,1) Fight The Risk of Cancer!\n\nhttp://www.adc...,spam
emails/spam\00004.eac8de8d759b7e74154f142194282724,##############################################...,spam
emails/spam\00005.57696a39d7d84318ce497886896bf90d,I thought you might like these:\n\n1) Slim Dow...,spam
...,...,...
emails/ham\02497.60497db0a06c2132ec2374b2898084d3,"Hi Gianni,\n\n\n\nA very good resource for thi...",ham
emails/ham\02498.09835f512f156da210efb99fcc523e21,Gianni Ponzi wrote:\n\n> I have a prob when tr...,ham
emails/ham\02499.b4af165650f138b10f9941f6cc5bce3c,Neale Pickett <neale@woozle.org> writes:\n\n\n...,ham
emails/ham\02500.05b3496ce7bca306bed0805425ec8621,"\n\nHi,\n\n\n\nI think you need to give us a l...",ham


In [6]:
data.tail(2)

,message,class
emails/ham\02500.05b3496ce7bca306bed0805425ec8621,"\n\nHi,\n\n\n\nI think you need to give us a l...",ham
emails/ham\99999,,ham


In [7]:
data.head(2)

,message,class
emails/spam\00001.7848dde101aa985090474a91ec93fcf0,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 Tr...",spam
emails/spam\00002.d94f1b97e48ed3b553b3508d116e6a09,1) Fight The Risk of Cancer!\n\nhttp://www.adc...,spam


In [8]:
data.shape

(3001, 2)

In [9]:
data['class'].value_counts()

ham     2501
spam     500
Name: class, dtype: int64

In [10]:
# We will use a CountVectorizer to split up each message into it's list of word, and throw that into Naive Bayes classifier. We will call fit() and get trained smap classifier

vectorizer = CountVectorizer()
counts = vectorizer.fit_transform( data['message'].values)

counts


<3001x62964 sparse matrix of type '<class 'numpy.int64'>'
	with 429785 stored elements in Compressed Sparse Row format>

In [11]:
# Build Naive Bayes model
clasifier = MultinomialNB()




In [12]:
# Train the Naive Bayes model
targets = data['class'].values

In [13]:

clasifier.fit(counts, targets)





MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [14]:
# Predictions

example = ['Free Viagra now!!!', 'Hello Dilip, today is workding day in Pune']
example_counts = vectorizer.transform(example)
predictions = clasifier.predict(example_counts)
predictions

array(['spam', 'ham'], dtype='<U4')

In [17]:
# Predictions
example = ['Yesterday, it was raining heavily in Mumbai!',\
           'We are fortune 500 company, you won lottery','My name is surbhi and  i like singing','you received lottery']
example_counts = vectorizer.transform(example)
predictions = clasifier.predict(example_counts)
predictions



array(['ham', 'spam', 'ham', 'spam'], dtype='<U4')